In [1]:
# Import necessary packages
import pandas as pd
from pymongo import MongoClient
import time
import psutil
import cudf
from extract_load_functions_mongodb import extraction, loading

#set export location
exportLocation = r'/home/jeff/'

#set number of test runs to prefrom
iterations = 31

# Connect to MongoDb
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ds7330 
item = db.item


etlTimerStart = time.perf_counter() 
# LIGHT EXTRACTION #
### Pull item table and capture CPU, RAM and elapsed time to perform operations
                               
lightExtraction = extraction(item, iterations, exportLocation, 'lightExtractionPrfm_Mongo.csv')
  

#### LIGHT TRANSFORMATION ####
###################################################################################################################
# Preform light transformation workload.  In this case, identify all item descriptions with "Blue" in them 
# and change them to "Navy"

    #Cast datatypes to objects

column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
lightTransPrfm = pd.DataFrame(columns = column_names)
#lightTrans.item_desc.str.contains("^Blue")

print("STARTING TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

transTimerStart = time.perf_counter()
#Run 30 iterations to collect transformation df of performance metrics
for sampleNoTransform in range(iterations):
    convert_dict = {'_id': str, 
            'orderedItem': str,
            'item_desc':str,
            'selling_price':float
           } 
    lightExtraction = lightExtraction.astype(convert_dict) 
    
    lightTrans = pd.DataFrame(lightExtraction)
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    lightTrans.drop(lightTrans.columns[[0]], axis = 1, inplace = True) 
    lightTrans.drop([406], axis = 0, inplace = True)
    lightTrans.item_desc = lightTrans.item_desc.str.replace('Blue', 'Navy', regex=True)
    #Stop timer  
    stop = time.perf_counter()
    
    
    lightTransPrfm = lightTransPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
    #lightTrans = orderItemJoin
    time.sleep(2)
transTimerEnd= time.perf_counter() 

print("Pandas transformation metrics captured.\n")
print("Total extraction time: ", transTimerEnd - transTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average extraction iteration time: ", lightTransPrfm.elapsed_time.mean(), "s")
print("Average CPU utilization: ", lightTransPrfm.CPU_utilization.mean())
print("Average RAM utilization: ", lightTransPrfm.RAM_utilization.mean(), "\n\n")

# Export Light Transformation Performance to local
lightTransPrfm.to_csv (r'/home/jeff/lightTransPrfm_mongodb.csv', index = False, header=True)


#### cuDF TRANSFORMATION ####
###################################################################################################################

lightTrans2 = lightExtraction
column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
lightTransPrfmCU = pd.DataFrame(columns = column_names)

print("Starting cuDF TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

#Run 30 iterations to collect transformation df of performance metrics
transCuTimerStart = time.perf_counter()
for sampleNoTransform in range(iterations):
    ## Transform pandas df to cuDF
    lightTransCU = cudf.DataFrame.from_pandas(lightTrans2)
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    lightTransCU.drop_column("_id")
    lightTransCU = lightTransCU[0:406]
    lightTransCU.item_desc = lightTransCU.item_desc.str.replace('Blue', 'Navy', regex=True)
    #Stop timer  
    stop = time.perf_counter()
    
    lightTransPrfmCU = lightTransPrfmCU.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
    #lightTrans = orderItemJoin
    time.sleep(2)
transCuTimerEnd = time.perf_counter() 

print("cuDF performance metrics captured loading complete.\n")
print("Total cuDF TRANSFORMATION time: ", transCuTimerEnd - transCuTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average cuDF TRANSFORMATION iteration time: ", lightTransPrfmCU.elapsed_time.mean(), "s")
print("Average cuDF TRANSFORMATION CPU utilization: ", lightTransPrfmCU.CPU_utilization.mean())
print("Average cuDF TRANSFORMATION RAM utilization: ", lightTransPrfmCU.RAM_utilization.mean(),"\n\n")

# Export Light Transformation cuDF Performance to local
lightTransPrfmCU.to_csv (r'/home/jeff/lightTransPrfmCU_mongodb.csv', index = False, header=True)

#### Load data ####
###################################################################################################################
loading(myclient,'light', lightTrans, iterations, exportLocation, "lightMongoDBLoad.csv")

etlTimerEnd = time.perf_counter() 
print('ETL is complete')
print('Elapsed ETL time is: ', (etlTimerEnd-etlTimerStart)/60, ' minutes')

STARTING EXTRACTION...
Base CPU utilization:  6.3
Base RAM utilization:  16.1
RUNNING...
Data frame loading complete.

Total EXTRACTION time:  62.22649265600194 s
Iterations performed:  31
Average EXTRACTION iteration time:  0.004271541160696577 s
Average CPU utilization:  3.0548387096774188
Average RAM utilization:  16.264516129032266 


STARTING TRANSFORMATION...
Base CPU utilization:  2.2
Base RAM utilization:  16.3
RUNNING...
Pandas transformation metrics captured.

Total extraction time:  62.215695251004945 s
Iterations performed:  31
Average extraction iteration time:  0.0020747309031468185 s
Average CPU utilization:  2.2838709677419358
Average RAM utilization:  16.303225806451618 


Starting cuDF TRANSFORMATION...
Base CPU utilization:  2.2
Base RAM utilization:  16.3
RUNNING...
cuDF performance metrics captured loading complete.

Total cuDF TRANSFORMATION time:  62.986619641997095 s
Iterations performed:  31
Average cuDF TRANSFORMATION iteration time:  0.011228709805788727 s
Av